In [ ]:
!python --version

Python 3.7.12


## Setup Environment

1/2 Spark

In [ ]:
!pwd
# get spark v3
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

/content
--2021-10-31 21:47:56--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M   196MB/s    in 1.1s    

2021-10-31 21:47:57 (196 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]



In [ ]:
!tar -xvzf spark-3.1.2-bin-hadoop3.2.tgz

spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.Rmd
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/index.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.1.2-

In [ ]:
!ls /content/spark-3.1.2-bin-hadoop3.2

bin   data	jars	    LICENSE   NOTICE  R		 RELEASE  yarn
conf  examples	kubernetes  licenses  python  README.md  sbin


In [ ]:
!pip install findspark

2/2 Setup GeoPandas

*needed to process geospatial attributes.*

In [ ]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,818 kB/s)
Selecting previously unselected

## Read NYC taxi dataset

Initialize a Spark session

In [ ]:
import os
os.environ['SPARK_HOME'] = "/content/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark").getOrCreate()

Read a sample of the NYC taxi dataset from http://www.andresmh.com/nyctaxitrips/

In [ ]:
# url = https://github.com/Nnawar7/Analyzing-NYC-Taxi/blob/main/sample.csv
file_location = "/content/sample.csv"
file_type="csv"
taxi_df = spark.read\
                .format(file_type)\
                .option("header", "true")\
                .load(file_location)

## Explore dataset

Dump first n rows

In [ ]:
taxi_df.show(2)

+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|    pickup_datetime|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+-------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      CMT|        1|                 N|2013-01-01 15:11:48|2013-01-01 15:18:10|              4|              382|         1.00|      -73.978165|      40.757977|       -73.989838|       40.751171|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...| 

Print schema

*Notice co-ordinates string data-type.*

In [ ]:
taxi_df.printSchema() #longitude and latitude are strings

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



Print dataset size

In [ ]:
taxi_df.count()

99999

Checking for nulls 

In [ ]:
import pyspark.sql.functions as F
for col_name in taxi_df.columns:
    print("The count of null rows in column %s" %col_name, taxi_df.where(F.col(col_name).isNull()).count())

The count of null rows in column medallion 0
The count of null rows in column hack_license 0
The count of null rows in column vendor_id 0
The count of null rows in column rate_code 0
The count of null rows in column store_and_fwd_flag 99626
The count of null rows in column pickup_datetime 0
The count of null rows in column dropoff_datetime 0
The count of null rows in column passenger_count 0
The count of null rows in column trip_time_in_secs 0
The count of null rows in column trip_distance 0
The count of null rows in column pickup_longitude 0
The count of null rows in column pickup_latitude 0
The count of null rows in column dropoff_longitude 0
The count of null rows in column dropoff_latitude 0


Checking for zeros

*Notice trips with zero time/ pickup location/ dropoff location/ passengers.*

In [ ]:
for col_name in taxi_df.columns:
    print("The count of zero rows in column %s" %col_name, taxi_df.where(F.col(col_name) == "0").count())

The count of zero rows in column medallion 0
The count of zero rows in column hack_license 0
The count of zero rows in column vendor_id 0
The count of zero rows in column rate_code 0
The count of zero rows in column store_and_fwd_flag 0
The count of zero rows in column pickup_datetime 0
The count of zero rows in column dropoff_datetime 0
The count of zero rows in column passenger_count 1
The count of zero rows in column trip_time_in_secs 449
The count of zero rows in column trip_distance 0
The count of zero rows in column pickup_longitude 1770
The count of zero rows in column pickup_latitude 1749
The count of zero rows in column dropoff_longitude 1917
The count of zero rows in column dropoff_latitude 1901


## Prepare dataset

Drop rows with zero trip time/ co-ordinates/ passengers

In [ ]:
cols_with_zeros = ["passenger_count", "trip_time_in_secs", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]
for col_name in cols_with_zeros:
  print("Cleanup for: ", col_name)
  print("Count before cleanup: ", taxi_df.count())
  taxi_df = taxi_df.where(F.col(col_name) != "0")
  print("Count after cleanup: %s \n" % taxi_df.count())

Cleanup for:  passenger_count
Count before cleanup:  99999
Count after cleanup: 99998 

Cleanup for:  trip_time_in_secs
Count before cleanup:  99998
Count after cleanup: 99549 

Cleanup for:  pickup_longitude
Count before cleanup:  99549
Count after cleanup: 98043 

Cleanup for:  pickup_latitude
Count before cleanup:  98043
Count after cleanup: 98042 

Cleanup for:  dropoff_longitude
Count before cleanup:  98042
Count after cleanup: 97891 

Cleanup for:  dropoff_latitude
Count before cleanup:  97891
Count after cleanup: 97888 



In [ ]:
for col_name in taxi_df.columns:
    print("The count of zero rows in column %s" %col_name, taxi_df.where(F.col(col_name) == "0").count())

The count of zero rows in column medallion 0
The count of zero rows in column hack_license 0
The count of zero rows in column vendor_id 0
The count of zero rows in column rate_code 0
The count of zero rows in column store_and_fwd_flag 0
The count of zero rows in column pickup_datetime 0
The count of zero rows in column dropoff_datetime 0
The count of zero rows in column passenger_count 0
The count of zero rows in column trip_time_in_secs 0
The count of zero rows in column trip_distance 0
The count of zero rows in column pickup_longitude 0
The count of zero rows in column pickup_latitude 0
The count of zero rows in column dropoff_longitude 0
The count of zero rows in column dropoff_latitude 0


In [ ]:
taxi_df.count()

97888

Drop unnecessary features

In [ ]:
taxi_df=taxi_df.drop("medallion","vendor_id","rate_code","store_and_fwd_flag","passenger_count","trip_distance")

Cast longitude and latitude to float

In [ ]:
from pyspark.sql.types import FloatType
taxi_df= taxi_df.withColumn("pickup_latitude", taxi_df["pickup_latitude"].cast(FloatType()))\
     .withColumn("pickup_longitude",taxi_df["pickup_longitude"].cast(FloatType()))\
     .withColumn("dropoff_longitude",taxi_df["dropoff_longitude"].cast(FloatType()))\
     .withColumn("dropoff_latitude",taxi_df["dropoff_latitude"].cast(FloatType()))

In [ ]:
taxi_df.printSchema()

root
 |-- hack_license: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)



## Read GeoJson data

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame # Loading boundaries Data
from shapely.geometry import Point, Polygon, shape, LineString

In [ ]:
geo_df = GeoDataFrame.from_file('/content/nyc-boroughs.geojson')

## Explore GeoJson data

In [ ]:
geo_df.head(5)

,boroughCode,borough,@id,geometry
0,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-74.05051 40.56642, -74.04998 40.566..."
1,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-74.05314 40.57770, -74.05406 40.577..."
2,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-74.15946 40.64145, -74.15998 40.641..."
3,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-74.08221 40.64828, -74.08142 40.648..."
4,4,Queens,http://nyc.pediacities.com/Resource/Borough/Qu...,"POLYGON ((-73.83668 40.59495, -73.83671 40.594..."


In [ ]:
geo_df['geometry']

0      POLYGON ((-74.05051 40.56642, -74.04998 40.566...
1      POLYGON ((-74.05314 40.57770, -74.05406 40.577...
2      POLYGON ((-74.15946 40.64145, -74.15998 40.641...
3      POLYGON ((-74.08221 40.64828, -74.08142 40.648...
4      POLYGON ((-73.83668 40.59495, -73.83671 40.594...
                             ...                        
99     POLYGON ((-73.78103 40.87648, -73.78121 40.876...
100    POLYGON ((-73.78651 40.88094, -73.78582 40.880...
101    POLYGON ((-73.87295 40.90444, -73.85947 40.900...
102    POLYGON ((-73.80518 40.81527, -73.80508 40.815...
103    POLYGON ((-73.80408 40.81349, -73.80424 40.813...
Name: geometry, Length: 104, dtype: geometry

In [ ]:
geo_df.isnull().sum()

boroughCode    0
borough        0
@id            0
geometry       0
dtype: int64

In [ ]:
geo_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   boroughCode  104 non-null    int64   
 1   borough      104 non-null    object  
 2   @id          104 non-null    object  
 3   geometry     104 non-null    geometry
dtypes: geometry(1), int64(1), object(2)
memory usage: 3.4+ KB


In [ ]:
len(geo_df)

104

In [ ]:
geo_df_no_dups = geo_df.drop_duplicates(subset=['boroughCode','borough'])
geo_df_no_dups

,boroughCode,borough,@id,geometry
0,5,Staten Island,http://nyc.pediacities.com/Resource/Borough/St...,"POLYGON ((-74.05051 40.56642, -74.04998 40.566..."
4,4,Queens,http://nyc.pediacities.com/Resource/Borough/Qu...,"POLYGON ((-73.83668 40.59495, -73.83671 40.594..."
24,3,Brooklyn,http://nyc.pediacities.com/Resource/Borough/Br...,"POLYGON ((-73.86706 40.58209, -73.86815 40.581..."
45,1,Manhattan,http://nyc.pediacities.com/Resource/Borough/Ma...,"POLYGON ((-74.01093 40.68449, -74.01193 40.683..."
74,2,Bronx,http://nyc.pediacities.com/Resource/Borough/Bronx,"POLYGON ((-73.89681 40.79581, -73.89694 40.795..."


In [ ]:
len(geo_df), len(geo_df_no_dups)

(104, 5)

## Enrich Main dataset with GeoJson data

#### **Joining the dataframes**: 0-*geodataframes*

Convert main dataset dataframe to geodataframe


In [ ]:
def spark_df_to_geopandas(sdf):
  df = sdf.toPandas()
  gdf = gpd.GeoDataFrame(
    df.drop(["pickup_longitude", "pickup_latitude"], axis=1),
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(df.pickup_longitude, df.pickup_latitude)])
  return gdf

In [ ]:
nyc_gdf=spark_df_to_geopandas(taxi_df)

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Remove unnecessary columns

In [ ]:
nyc_gdf=nyc_gdf.drop(['dropoff_longitude','dropoff_latitude'],axis=1)

In [ ]:
geo_df=geo_df.drop(['@id'],axis=1)

Inner join via within condition

In [ ]:
enriched_via_geodataframe_join = nyc_gdf.sjoin((geo_df), how="inner", op="within")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1938: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [ ]:
enriched_via_geodataframe_join.head(5)

,hack_license,pickup_datetime,dropoff_datetime,trip_time_in_secs,geometry,index_right,boroughCode,borough
0,BA96DE419E711691B9445D6A6307C170,2013-01-01 15:11:48,2013-01-01 15:18:10,382,POINT (-73.97816 40.75798),72,1,Manhattan
1,9FD8F69F0804BDB5549F40E9DA1BE472,2013-01-06 00:18:35,2013-01-06 00:22:54,259,POINT (-74.00668 40.73178),72,1,Manhattan
2,9FD8F69F0804BDB5549F40E9DA1BE472,2013-01-05 18:49:41,2013-01-05 18:54:23,282,POINT (-74.00471 40.73777),72,1,Manhattan
3,51EE87E3205C985EF8431D850C786310,2013-01-07 23:54:15,2013-01-07 23:58:20,244,POINT (-73.97460 40.75994),72,1,Manhattan
4,51EE87E3205C985EF8431D850C786310,2013-01-07 23:25:03,2013-01-07 23:34:24,560,POINT (-73.97625 40.74853),72,1,Manhattan


In [ ]:
enriched_via_geodataframe_join['borough'].isnull().sum()

0

In [ ]:
enriched_via_geodataframe_join.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 97747 entries, 0 to 69802
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   hack_license       97747 non-null  object  
 1   pickup_datetime    97747 non-null  object  
 2   dropoff_datetime   97747 non-null  object  
 3   trip_time_in_secs  97747 non-null  object  
 4   geometry           97747 non-null  geometry
 5   index_right        97747 non-null  int64   
 6   boroughCode        97747 non-null  int64   
 7   borough            97747 non-null  object  
dtypes: geometry(1), int64(2), object(5)
memory usage: 6.7+ MB


#### **Joining the dataframes**: 1-*two-dimensional loop*


does the polygon contain the point?

In [ ]:
def contains(Point_X, Point_Y, Polygon):
  point = geometry.Point(Point_X, Point_Y)
  line = geometry.LineString(Polygon)
  polygon = geometry.Polygon(line)
  polygon.contains(point)

manually test shapely contains with specific point and polygon

In [ ]:
Point_X = -73.978165
Point_Y = 40.757977
point = Point(Point_X, Point_Y)
print(point)
print(geo_df['geometry'][0].contains(point))

POINT (-73.978165 40.757977)
False


loop on geoJson data inside another loop on main dataset 

*worst performance and it gets only pickup borough!*

In [ ]:
def get_pickup_borough(row):
  point = Point(float(row['pickup_longitude']), float(row['pickup_latitude']))
  borough = ""
  for key, polygon in enumerate(geo_df['geometry']):
    if (polygon.contains(point)):
      borough = geo_df["borough"][key]
      break
  # if (borough == ""):
  #   print("failed coordinates: ", row['pickup_longitude'], row['pickup_latitude'])
  return (row["pickup_longitude"], row["pickup_latitude"], borough)


enriched_2d = taxi_df.rdd.map(get_pickup_borough).toDF(["pickup_longitude", "pickup_latitude", "pickup_borough"])
enriched_2d.show(2)
# notice the empty borough in the last table, need to look into that further.
enriched_2d.groupby(['pickup_borough']).count().limit(5).show()

+------------------+------------------+--------------+
|  pickup_longitude|   pickup_latitude|pickup_borough|
+------------------+------------------+--------------+
|-73.97816467285156| 40.75797653198242|     Manhattan|
|-74.00668334960938|40.731781005859375|     Manhattan|
+------------------+------------------+--------------+
only showing top 2 rows

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|        Queens| 5840|
|      Brooklyn| 1958|
| Staten Island|    2|
|              |  141|
|     Manhattan|89868|
+--------------+-----+



In [ ]:
enriched_2d.where(enriched_2d["pickup_borough"] == "").show(5)

+------------------+------------------+--------------+
|  pickup_longitude|   pickup_latitude|pickup_borough|
+------------------+------------------+--------------+
|-74.10505676269531|40.655216217041016|              |
|-74.15985870361328| 40.70987319946289|              |
|-74.16817474365234|40.712398529052734|              |
|-73.92723846435547| 40.80000686645508|              |
| -73.7412338256836|40.787296295166016|              |
+------------------+------------------+--------------+
only showing top 5 rows



#### **Tune Spark**

List spark parameters

In [ ]:
print("Spark parameters:")
print("spark.sparkContext.defaultParallelism: ", spark.sparkContext.defaultParallelism)
print("spark.sql.files.maxPartitionBytess: ", spark.conf.get('spark.sql.files.maxPartitionBytes'))
print("spark.sql.files.maxRecordsPerFile: ", spark.conf.get('spark.sql.files.maxRecordsPerFile'))
print("spark.sql.shuffle.partitions: ", spark.conf.get('spark.sql.shuffle.partitions'))

Spark parameters:
spark.sparkContext.defaultParallelism:  2
spark.sql.files.maxPartitionBytess:  134217728b
spark.sql.files.maxRecordsPerFile:  0
spark.sql.shuffle.partitions:  200


List dataframe properties

In [ ]:
print("Dataframe properties:")
print("numPartitions: ", enriched_2d.rdd.getNumPartitions())

Dataframe properties:
numPartitions:  2


*Notice, using all available partitions, now let's check skewness.*

In [ ]:
from pyspark.sql.functions import spark_partition_id

def show_part_dist():
  enriched_2d.select(spark_partition_id().alias("partitionId")).groupBy("partitionId").count().show()

In [ ]:
show_part_dist()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          1|36567|
|          0|61321|
+-----------+-----+



Create a benchmark test

In [ ]:
import time
def run_test_query():
  start_time = time.perf_counter()
  enriched_2d.groupby(['pickup_borough']).count().limit(5).show()
  end_time = time.perf_counter()
  time_elapsed = (round(end_time) - round(start_time))
  print("Elapsed time (secs): ", time_elapsed)

Benchmark default performance

In [ ]:
run_test_query()

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|        Queens| 5840|
|      Brooklyn| 1958|
| Staten Island|    2|
|              |  141|
|     Manhattan|89868|
+--------------+-----+

Elapsed time (secs):  95


Repartition to fix data skewness

In [ ]:
enriched_2d = enriched_2d.repartition(2)

In [ ]:
show_part_dist()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          1|48943|
|          0|48945|
+-----------+-----+



Benchmark repartition

*Notice insignificant effect*

In [ ]:
run_test_query()

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|        Queens| 5840|
|      Brooklyn| 1958|
| Staten Island|    2|
|              |  141|
|     Manhattan|89868|
+--------------+-----+

Elapsed time (secs):  96


Limit shuffle partitions to available ones

In [ ]:
spark.conf.set('spark.sql.shuffle.partitions', 2)

Benchmark shuffle partitions

*Notice insignificant effect*

In [ ]:
run_test_query()

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|              |  141|
| Staten Island|    2|
|        Queens| 5840|
|     Manhattan|89868|
|      Brooklyn| 1958|
+--------------+-----+

Elapsed time (secs):  95


Set serializer as Kryo serializer

Throws *AnalysisException: Cannot modify the value of a Spark config: spark.serializer*

In [ ]:
## AnalysisException: Cannot modify the value of a Spark config: spark.serializer
# spark.conf.set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')

Convert to parquet

In [ ]:
enriched_2d.write.mode("overwrite").parquet("/content/enriched_2d")

In [ ]:
enriched_2d = spark.read.parquet("/content/enriched_2d")

Benchmark parquet

*Notice huge effect*

In [ ]:
run_test_query()

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|        Queens| 5840|
|              |  141|
| Staten Island|    2|
|     Manhattan|89868|
|      Brooklyn| 1958|
+--------------+-----+

Elapsed time (secs):  0


Convert main dataset to parquet

In [ ]:
taxi_df.write.mode("overwrite").parquet("/content/sample")

In [ ]:
!zip -r /content/sample.zip /content/sample
# uploaded at https://drive.google.com/file/d/1IlsIgLxKvHfhtJT1nR3ChOnQ1_8WWGn0/view?usp=sharing

  adding: content/sample/ (stored 0%)
  adding: content/sample/part-00000-ea57cda4-a6a3-498c-af15-f74c30f7be9f-c000.snappy.parquet (deflated 14%)
  adding: content/sample/._SUCCESS.crc (stored 0%)
  adding: content/sample/.part-00000-ea57cda4-a6a3-498c-af15-f74c30f7be9f-c000.snappy.parquet.crc (stored 0%)
  adding: content/sample/.part-00001-ea57cda4-a6a3-498c-af15-f74c30f7be9f-c000.snappy.parquet.crc (stored 0%)
  adding: content/sample/part-00001-ea57cda4-a6a3-498c-af15-f74c30f7be9f-c000.snappy.parquet (deflated 16%)
  adding: content/sample/_SUCCESS (stored 0%)


In [ ]:
taxi_df = spark.read.parquet("/content/sample")

Dump first n rows

In [ ]:
taxi_df.show(2)

+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              382|      -73.978165|      40.757977|        -73.98984|        40.75117|
|9FD8F69F0804BDB55...|2013-01-06 00:18:35|2013-01-06 00:22:54|              259|       -74.00668|       40.73178|         -73.9945|        40.75066|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+
only showing top 2 rows



Print schema

*Notice co-ordinates float data-type (as expected).*

In [ ]:
taxi_df.printSchema() #longitude and latitude are float

root
 |-- hack_license: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)



Print dataset size

In [ ]:
taxi_df.count()

97888

#### **Tune GeoJson Data**

Sort geojson data via boroughCode and area

In [ ]:
geo_df['area'] = geo_df.geometry.area
geo_df_sorted = geo_df.sort_values(['boroughCode', 'area'], ascending=[True, False])
geo_df_sorted.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


,boroughCode,borough,geometry,area
72,1,Manhattan,"POLYGON ((-73.92641 40.87762, -73.92630 40.877...",0.005859
71,1,Manhattan,"POLYGON ((-73.92134 40.80085, -73.92031 40.799...",0.000233
51,1,Manhattan,"POLYGON ((-74.01675 40.69334, -74.01540 40.693...",0.000076
69,1,Manhattan,"POLYGON ((-73.94180 40.76905, -73.94286 40.768...",0.000062
73,1,Manhattan,"POLYGON ((-73.90665 40.87575, -73.90774 40.872...",0.000033
57,1,Manhattan,"POLYGON ((-74.03995 40.70089, -74.03771 40.699...",0.000012
49,1,Manhattan,"POLYGON ((-74.04388 40.69019, -74.04351 40.689...",0.000006
58,1,Manhattan,"POLYGON ((-73.99538 40.70264, -73.99670 40.700...",0.000003
50,1,Manhattan,"POLYGON ((-74.00133 40.69123, -74.00070 40.690...",0.000003
53,1,Manhattan,"POLYGON ((-74.00078 40.69429, -74.00096 40.694...",0.000002


Broadcast geojson data

In [ ]:
geo_bc = spark.sparkContext.broadcast(geo_df_sorted)

#### **Joining the dataframes**: 2- *map + spatial join*

In [ ]:
## Using a map through rows
interesting_exiting_cols = ["hack_license", "pickup_datetime", "dropoff_datetime", "trip_time_in_secs", "pickup_latitude", "pickup_longitude", "dropoff_longitude", "dropoff_latitude"]
interesting_new_cols = ["pickup_borough", "dropoff_borough"]
interesting_columns = interesting_exiting_cols + interesting_new_cols 
def get_borough(row):
  data = {'type': ['pickup', 'dropoff'], 'geometry': [Point(row.pickup_longitude, row.pickup_latitude), Point(row.dropoff_longitude, row.dropoff_latitude)]}
  data_gdf = gpd.GeoDataFrame(data, crs={'init': 'epsg:4326'})
  geo_gdf = geo_bc.value
  joined_df = data_gdf.sjoin(geo_gdf)
  pickup_borough = ""
  dropoff_borough = ""
  boroughs = joined_df.borough
  if (0 in boroughs.index):
    pickup_borough = boroughs[0]
  if (1 in boroughs.index):
    dropoff_borough = boroughs[1]
  return [row[col] for col in interesting_exiting_cols] + [pickup_borough, dropoff_borough]

Enrich via map

In [ ]:
enriched_df = taxi_df.rdd.map(get_borough).toDF(interesting_columns)
enriched_df.show(5)

+--------------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+--------------+---------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|   pickup_latitude|  pickup_longitude| dropoff_longitude|  dropoff_latitude|pickup_borough|dropoff_borough|
+--------------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+------------------+--------------+---------------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              382| 40.75797653198242|-73.97816467285156|-73.98983764648438| 40.75117111206055|     Manhattan|      Manhattan|
|9FD8F69F0804BDB55...|2013-01-06 00:18:35|2013-01-06 00:22:54|              259|40.731781005859375|-74.00668334960938|-73.99449920654297| 40.75065994262695|     Manhattan|      Manhattan|
|9FD8F69F0804BDB55...|2013-01-05 18:49:41|2013-01-05 18:54:2

#### **Joining the dataframes**: 3- *UDF + spatial join*

Investigate pandas_udf type of user-defined functions

In [ ]:
from pyspark.sql.functions import pandas_udf

def udf_schema_from_pandas(df):
  column_types  = [StructField(key, mapping[dict(df.dtypes)[key]]()) for key in df.columns]
  schema = StructType(column_types)
  return schema

# @pandas_udf(schema_out, PandasUDFType.GROUPED_MAP)
def join_nyc_geo(nyc):
    gdf_geo = gpd.GeoDataFrame(geo, geometry=geometry, crs=crs )
    gdf_nyc = gpd.GeoDataFrame(nyc , geometry=geometry, crs=crs)
    joined_df = gpd.sjoin(gdf_nyc, gdf_geo, how='inner', op='within')
    return joined_df[columns]

    # Need to look on how to return a schema from UDF

Prepare a traditional user-defined function

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, StringType

def get_borough_UDF(pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude):
  data = {'type': ['pickup', 'dropoff'], 'geometry': [Point(pickup_longitude, pickup_latitude), Point(dropoff_longitude, dropoff_latitude)]}
  data_gdf = gpd.GeoDataFrame(data, crs={'init': 'epsg:4326'})
  geo_gdf = geo_bc.value
  joined_df = data_gdf.sjoin(geo_gdf)
  pickup_borough = ""
  dropoff_borough = ""
  boroughs = joined_df.borough
  if (0 in boroughs.index):
    pickup_borough = boroughs[0]
  if (1 in boroughs.index):
    dropoff_borough = boroughs[1]
  return [pickup_borough,dropoff_borough]
convertUDF = udf(get_borough_UDF, ArrayType(StringType()))

Test enrichment on a single row

In [ ]:
row = taxi_df.first()
print(get_borough_UDF(row.pickup_longitude,row.pickup_latitude,row.dropoff_longitude,row.dropoff_latitude))

['Manhattan', 'Manhattan']


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1938: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


Enrich via UDF

In [ ]:
enriched_udf = taxi_df.withColumn("udfResult",convertUDF('pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'))\
  .withColumn("pickup_borough", col("udfResult")[0])\
  .withColumn("dropoff_borough", col("udfResult")[1])\
  .drop("udfResult")
enriched_udf.show(5)

+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_borough|dropoff_borough|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              382|      -73.978165|      40.757977|        -73.98984|        40.75117|     Manhattan|      Manhattan|
|9FD8F69F0804BDB55...|2013-01-06 00:18:35|2013-01-06 00:22:54|              259|       -74.00668|       40.73178|         -73.9945|        40.75066|     Manhattan|      Manhattan|
|9FD8F69F0804BDB55...|2013-01-05 18:49:41|2013-01-05 18:54:23|              282|       -74.00471|   

Convert to parquet partitioned by boroughs

Very effective performance optimization.

In [ ]:
enriched_udf.write.mode("overwrite").partitionBy("pickup_borough", "dropoff_borough").parquet("/content/enriched_udf")

In [ ]:
!zip -r /content/enriched_udf.zip /content/enriched_udf
# uploaded at https://drive.google.com/file/d/1u6E91nFnvBQLZD2r5DMVClC0ah-v8nWe/view?usp=sharing


  adding: content/enriched_udf/ (stored 0%)
  adding: content/enriched_udf/pickup_borough=__HIVE_DEFAULT_PARTITION__/ (stored 0%)
  adding: content/enriched_udf/pickup_borough=__HIVE_DEFAULT_PARTITION__/dropoff_borough=Manhattan/ (stored 0%)
  adding: content/enriched_udf/pickup_borough=__HIVE_DEFAULT_PARTITION__/dropoff_borough=Manhattan/part-00000-9afa403f-3af6-4f16-9f24-e7b00207810f.c000.snappy.parquet (deflated 43%)
  adding: content/enriched_udf/pickup_borough=__HIVE_DEFAULT_PARTITION__/dropoff_borough=Manhattan/.part-00000-9afa403f-3af6-4f16-9f24-e7b00207810f.c000.snappy.parquet.crc (stored 0%)
  adding: content/enriched_udf/pickup_borough=__HIVE_DEFAULT_PARTITION__/dropoff_borough=Manhattan/part-00001-9afa403f-3af6-4f16-9f24-e7b00207810f.c000.snappy.parquet (deflated 51%)
  adding: content/enriched_udf/pickup_borough=__HIVE_DEFAULT_PARTITION__/dropoff_borough=Manhattan/.part-00001-9afa403f-3af6-4f16-9f24-e7b00207810f.c000.snappy.parquet.crc (stored 0%)
  adding: content/enriched

In [ ]:
enriched_udf = spark.read.parquet("/content/enriched_udf")

Analyze borough distribution

In [ ]:
enriched_udf.groupby(['pickup_borough']).count().show()
enriched_udf.groupby(['dropoff_borough']).count().show()

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|        Queens| 5840|
|          null|  141|
| Staten Island|    2|
|     Manhattan|89868|
|      Brooklyn| 1958|
|         Bronx|   79|
+--------------+-----+

+---------------+-----+
|dropoff_borough|count|
+---------------+-----+
|         Queens| 5443|
|           null|  425|
|  Staten Island|   13|
|      Manhattan|88025|
|       Brooklyn| 3589|
|          Bronx|  393|
+---------------+-----+



## Detect outliers

Drop rows with undefined borough

In [ ]:
for col_name in ["pickup_borough", "dropoff_borough"]:
  enriched_udf = enriched_udf.where(F.col(col_name) != "")

enriched_udf.groupby(['pickup_borough']).count().limit(5).show()
enriched_udf.groupby(['dropoff_borough']).count().limit(5).show()

+--------------+-----+
|pickup_borough|count|
+--------------+-----+
|        Queens| 5765|
| Staten Island|    2|
|     Manhattan|89583|
|      Brooklyn| 1950|
|         Bronx|   76|
+--------------+-----+

+---------------+-----+
|dropoff_borough|count|
+---------------+-----+
|         Queens| 5430|
|  Staten Island|   12|
|      Manhattan|87959|
|       Brooklyn| 3582|
|          Bronx|  393|
+---------------+-----+



Drop rows wtih duration < 0 or > 4 hours

In [ ]:
from pyspark.sql.functions import unix_timestamp, col, to_date, date_format,from_unixtime,udf,hour,percentile_approx

enriched_udf = enriched_udf.withColumn ("pickup_timestamp", unix_timestamp(col("pickup_datetime")))\
        .withColumn ("dropoff_timestamp", unix_timestamp(col("dropoff_datetime")))

taxi_df = taxi_df.withColumn ("pickup_timestamp", unix_timestamp(col("pickup_datetime")))\
        .withColumn ("dropoff_timestamp", unix_timestamp(col("dropoff_datetime")))

In [ ]:
enriched_udf = enriched_udf.withColumn('Duration_ms', (F.col('dropoff_timestamp') - F.col('pickup_timestamp'))*1000)
taxi_df = taxi_df.withColumn('Duration_hrs',  (F.col('dropoff_timestamp') - F.col('pickup_timestamp'))/3600)

*Notice minimum is 0.001 Hr and maximum is 2.55 Hr, both are within limits (no outliers found).*

In [ ]:
result_pdf = taxi_df.select("*").toPandas()
result_pdf['Duration_hrs'].describe()

count    97888.000000
mean         0.181679
std          0.130190
min          0.001111
25%          0.100000
50%          0.150000
75%          0.233333
max          2.550000
Name: Duration_hrs, dtype: float64

In [ ]:
enriched_udf.show(5)

+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_borough|dropoff_borough|pickup_timestamp|dropoff_timestamp|Duration_ms|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+
|BA96DE419E711691B...|2013-01-01 15:11:48|2013-01-01 15:18:10|              382|      -73.978165|      40.757977|        -73.98984|        40.75117|     Manhattan|      Manhattan|      1357053108|       1357053490|     382000|
|9FD8F69F0804BDB55...|2013-01-06 00:18:35|2013-01-06 00:22:54|              259|       -74.0

In [ ]:
enriched_udf.where(F.col('Duration_ms') == 0).count()

0

In [ ]:
enriched_udf.count()

97376

## Queries

Window  Function to group by license, order by pickup datetime

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col, asc

windowSpec = Window\
             .partitionBy('hack_license')\
             .orderBy(asc('pickup_datetime'))

Add lag holding previous row's dropoff timestamp

In [ ]:
enriched_lag = enriched_udf.withColumn("lag",F.lag(enriched_udf['dropoff_timestamp'],offset=1).over(windowSpec))

enriched_lag.show(20)

+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_borough|dropoff_borough|pickup_timestamp|dropoff_timestamp|Duration_ms|       lag|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+
|0025133AD810DBE80...|2013-01-13 02:37:00|2013-01-13 02:46:00|              540|      -74.003914|       40.74254|       -73.984604|       40.769596|     Manhattan|      Manhattan|      1358044620|       1358045160|     540000|      null|
|0025133AD810DBE80...|2013-01-13 03:26:00|2013-0

Add idle time as difference between current pick time and last drop off time

In [ ]:
result = enriched_lag.withColumn('idle_time_secs', 
          (enriched_lag['pickup_timestamp'] - enriched_lag['lag']))

In [ ]:
result.show()

+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+--------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_borough|dropoff_borough|pickup_timestamp|dropoff_timestamp|Duration_ms|       lag|idle_time_secs|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+--------------+
|0025133AD810DBE80...|2013-01-13 02:37:00|2013-01-13 02:46:00|              540|      -74.003914|       40.74254|       -73.984604|       40.769596|     Manhattan|      Manhattan|      1358044620|       1358045160|     540000|      null|    

Filter trips with idle time > 4 hours 

In [ ]:
filtered_result = result.filter( (result.idle_time_secs < 14400) & (result.idle_time_secs.isNotNull()))

In [ ]:
filtered_result.count()

86003

Query 1: The average time it takes for a taxi to find its next trip per destination borough

In [ ]:
from pyspark.sql.functions import expr

def group_idle_time(groupBy):
  avg = expr("round(avg(idle_time_secs) / 3600, 2)").alias("average_idle_time")
  return filtered_result.groupBy(groupBy).agg(avg).orderBy(col("average_idle_time").desc())

df_grouped_dropoff_borough = group_idle_time("dropoff_borough")
df_grouped_dropoff_borough.show(10)

+---------------+-----------------+
|dropoff_borough|average_idle_time|
+---------------+-----------------+
|         Queens|             0.48|
|          Bronx|             0.46|
|       Brooklyn|             0.45|
|  Staten Island|             0.36|
|      Manhattan|             0.27|
+---------------+-----------------+



Query 2: The number of trips that started and ended within the same borough

In [ ]:
# def count_same_borough(df,count):
#   for row in range(df.shape[0]):
#    if (df['pickup_borough'][row] == df['dropoff_borough'][row]):
#       count+=1
#     return count

In [ ]:
same_borough = filtered_result.where(F.col('pickup_borough') == F.col('dropoff_borough'))
same_borough.show(5)

+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+--------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_borough|dropoff_borough|pickup_timestamp|dropoff_timestamp|Duration_ms|       lag|idle_time_secs|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+--------------+
|0025133AD810DBE80...|2013-01-13 03:26:00|2013-01-13 03:41:00|              900|       -74.00642|      40.744495|       -73.986916|        40.71598|     Manhattan|      Manhattan|      1358047560|       1358048460|     900000|1358045160|    

In [ ]:
same_borough_count = same_borough.count()
print("The number of trips that started and ended within the same borough is: ", same_borough_count)

The number of trips that started and ended within the same borough is:  77523


Query 3: The number of trips that started in one borough and ended in another


In [ ]:
different_borough = filtered_result.where(F.col('pickup_borough') != F.col('dropoff_borough'))
different_borough.show(5)

+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+--------------+
|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_borough|dropoff_borough|pickup_timestamp|dropoff_timestamp|Duration_ms|       lag|idle_time_secs|
+--------------------+-------------------+-------------------+-----------------+----------------+---------------+-----------------+----------------+--------------+---------------+----------------+-----------------+-----------+----------+--------------+
|002C093A2CB9FD40C...|2013-01-13 10:48:00|2013-01-13 11:06:00|             1080|       -73.96205|       40.77378|        -73.94704|       40.722702|     Manhattan|       Brooklyn|      1358074080|       1358075160|    1080000|1358073660|    

In [ ]:
different_borough_count = different_borough.count()
print("The number of trips that started and ended in different boroughs is: ", different_borough_count)

The number of trips that started and ended in different boroughs is:  8480
